In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf

In [4]:
from pydub import AudioSegment
from pydub.playback import play
import simpleaudio as sa
import serial


c:\Users\manya\Anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
model = tf.keras.models.load_model('actionFinal.h5')

In [6]:
actions = np.array(['piano', 'guitar', 'violin', 'drums', 'saxophone'])

In [7]:
folder_path = '.\Audio_files'
instruments = np.array(['piano', 'guitar', 'violin', 'drums', 'saxophone'])

Helper functions

In [10]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

#Drawing landmarks and connections
def draw_styled_landmarks(image, results):
    #body pose
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) 
                             ) 
    #left hand
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    #right hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) #concatenate since we're going to use all for gesture recognition

colors = [(245,117,16), (117,245,16), (16,117,245), (200,5,50), (11,25,245)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    return output_frame

Main

In [20]:
def control_realtime_testing(key):
    global is_testing_running
    global is_testing_paused
    
    # Start testing (press 's')
    if key == ord('s'):
        is_testing_running = True
        is_testing_paused = False
    
    # Pause testing (press 'p')
    elif key == ord('p'):
        is_testing_running = False
        is_testing_paused = True
    
    # Stop testing (press 'q')
    elif key == ord('q'):
        is_testing_running = False
        is_testing_paused = False

# Initialize variables for real-time testing control
is_testing_running = False
is_testing_paused = False

In [11]:
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        
        # Show control instructions on the frame
        cv2.putText(frame, "Press 's' to start, 'p' to pause, 'q' to stop", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [12]:
chosen_instrument = sentence[-1]

In [13]:
chosen_instrument

'piano'

Play all notes for chosen instrument

In [23]:
instrument_path = os.path.join(folder_path, chosen_instrument)

for file_name in os.listdir(instrument_path):
    if file_name.endswith(".wav"):
        # Construct the full file path
        file_path = os.path.join(instrument_path, file_name)
        
        wave_obj = sa.WaveObject.from_wave_file(file_path)
        play_obj = wave_obj.play()
        play_obj.wait_done() 

Play notes according to sensor feedback

In [14]:
instrument_path = os.path.join(folder_path, chosen_instrument)

In [15]:
instrument_path

'.\\Audio_files\\piano'

In [17]:
os.listdir(instrument_path)

['1.wav', '2.wav', '3.wav', '4.wav', '5.wav']

In [73]:
file_name = os.listdir(instrument_path)[1-1]

In [74]:
file_name

'1.wav'

In [ ]:
file_path = os.path.join(instrument_path, file_name)

In [76]:
file_path

'.\\Audio_files\\piano\\1.wav'

In [20]:
def play_note(note):
    file_name = os.listdir(instrument_path)[note]
    file_path = os.path.join(instrument_path, file_name)
    wave_obj = sa.WaveObject.from_wave_file(file_path)
    play_obj = wave_obj.play()
    play_obj.wait_done()  # Wait until sound has finished playing

In [ ]:
ser = serial.Serial('COM5', 9600)

note = 0

while True:
    signal = ser.readline().strip().decode('utf-8')
    
    if signal == 'S1':
        print("Sensor 1 is touched")
        play_note(0)
    
    if signal == 'S2':
        print("Sensor 2 is touched")
        play_note(1)
    
    if signal == 'S3':
        print("Sensor 3 is touched")
        play_note(2)

    if signal == 'S4':
        print("Sensor 4 is touched")
        play_note(3)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
            break

In [16]:
cap.release()
cv2.destroyAllWindows()